In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import os, sys
from typing import *
import torch
import random
import copy

In [2]:
print(os.getcwd())

for i in range (3):
    os.chdir("..")
    
main_data_dir = os.getcwd() + "/Data set"

/home/thaobeo/git/HeartResearch/Experiment/Approach/Model_signal


In [3]:
data_dir = "/media/mountHDD2/khoibaocon"
print(os.listdir(data_dir))

['TrainingSet3', 'Label.csv', 'alldata', 'TrainingSet1', 'single_label.csv', 'TrainingSet2']


In [4]:
main_df = pd.read_csv(data_dir + "/Label.csv")
main_df.shape

(6877, 4)

In [5]:
single_main_df = main_df[main_df["Second_label"].isnull()]
single_main_df.shape

(6400, 4)

In [6]:
single_main_df.to_csv(main_data_dir + "/single_label.csv")
single_fns = single_main_df["Recording"].values.tolist()
print(len(single_fns))

6400


In [7]:
single_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in single_fns]
print(len(single_mat_paths))

6400


In [13]:
ratio = [0.8, 0.1, 0.1]

# random.seed(42)
# img_data_list = random.sample(img_data_list, k = 55000)

train_index = int(len(single_mat_paths)*ratio[0])
valid_index = int(len(single_mat_paths)*(ratio[0]+ratio[1]))

train_mat_paths = single_mat_paths[:train_index]
valid_mat_paths = single_mat_paths[train_index:valid_index]
test_mat_paths = single_mat_paths[valid_index:]

train_label = single_main_df.iloc[:train_index,:]
valid_label = single_main_df.iloc[train_index:valid_index,:]
test_label = single_main_df.iloc[valid_index:,:]

In [14]:
class HeartData(Dataset):
    def __init__(self, data_paths):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        data = loadmat(data_path)['ECG'][0][0][2]
        clip_data = data[:, 300:3000]

        filename = data_path.split("/")[-1].split(".")[0]
        label = single_main_df[single_main_df["Recording"] == filename]["First_label"].values.item()

        torch_data = torch.from_numpy(clip_data)

        return torch_data.float(), label-1

    def __len__(self):
        return len(self.data_paths)

In [16]:
train_ds = HeartData(train_mat_paths)
test_ds = HeartData(test_mat_paths)
valid_ds = HeartData(valid_mat_paths)

In [17]:
train_dl = DataLoader(train_ds, batch_size = 128, shuffle = True, pin_memory = True, num_workers = 48)
valid_dl = DataLoader(valid_ds, batch_size = 128, shuffle = True, pin_memory = True, num_workers = 48)
test_dl = DataLoader(test_ds, batch_size = 128, shuffle = True, pin_memory = True, num_workers = 48)

In [20]:
class Auto_Encoder(nn.Module):
    def __init__(self):
        super(Auto_Encoder, self).__init__()

        self.enc = nn.Sequential(    
            # 1 x 105 x 105
            nn.Conv1d(1, 64, 4, 2, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # 64 x 52 x 52
            nn.Conv1d(64, 64, 3, 2, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # 64 x 26 x 26
            nn.Conv1d(64, 128, 3, 2, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            # 128 x 13 x 13
            nn.Conv1d(128, 128, 3, 2, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),  
            # 128 x 7 x 7
        )

        self.dw_latent = nn.Sequential(
            nn.Flatten(1),
            nn.Linear(7 * 7 * 128, 256),
        )
        
        self.latent = nn.Linear(256, 256)
        
        self.up_latent = nn.Sequential(
            nn.Linear(128, 256),
            nn.Linear(256, 7 * 7 * 128),            
        )

        self.dec = nn.Sequential(
            # 128 x 7 x 7
            nn.ConvTranspose1d(128, 128, 3, 2, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            # 128 x 13 x 13
            nn.ConvTranspose1d(128, 64, 3, 2, 1, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # 64 x 26 x 26
            nn.ConvTranspose1d(64, 64, 3, 2, 1, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # 64 x 52 x 52
            nn.ConvTranspose1d(64, 1, 4, 2, 1, 1),
            # 1 x 105 x 105
        )

        self.act = nn.Sigmoid()


    def forward(self, x):
        enc = self.enc(x)
        lat = self.latent(self.dw_latent(enc))
        mu, lv = self.unwrap(lat)
        rec_lat = self.up_latent(self.reparam(mu, lv))
        dec = self.dec(rec_lat.view(-1, 128, 7, 7))
        
        return dec, mu, lv

    def unwrap(self, x):
        return torch.split(x, x.shape[1]//2, dim=1)

    def reparam(self, mu, lv):
        if self.training:
            std = torch.exp(0.5 * lv)
            eps = torch.randn_like(std)
            return mu + std * eps
        else:
            return mu

In [22]:
epoch = 100
lr = 0.0001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)
model = Auto_Encoder().to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch)
loss_fn_cls = nn.CrossEntropyLoss()
loss_fn_sig = nn.MSELoss()

In [24]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(train_dl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred_cls, res_sig = model(train_sig)
        loss_cls = loss_fn_cls(pred_cls, train_label)
        loss_sig = loss_fn_sig(res_sig, train_sig)
        loss_tot = loss_cls + loss_sig
        
        optimizer.zero_grad()
        loss_tot.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss_tot.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(valid_dl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred_cls, res_sig = model(valid_sig)
            loss_cls = loss_fn_cls(pred_cls, valid_label)
            loss_sig = loss_fn_sig(res_sig, valid_sig)
            loss_tot = loss_cls + loss_sig

            val_total_loss += loss_tot.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")

Epoch: 0


0it [00:00, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 1, 4], expected input[128, 12, 2700] to have 1 channels, but got 12 channels instead